# HyMOD
HYMOD is a parsimonious rainfall-runoff model based based on the theory of runoff yeild under infiltration excess. The model structure is illustrated on the figure below.
<left><img src="util/HyMOD_diagram_simple.png" width="700px">

#### Import libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ipywidgets import widgets

from util.HyMOD import hymod

## Model parameters

In [2]:
data = [["mm"  , 1   , 100 , "Maximum storage capacity"],
        ["-"   , 0   , 2   , "Degree of spatial variability of the c_max"],
        ["-"   , 0.01, 0.99, "Factor distributing slow and quick flows"],
        ["-", 0.01, 0.10, "Fractional discharge of the slow release reservoir"],
        ["-", 0.10, 0.99, "Fractional discharge of the quick release reservoirs"]]
model_param = pd.DataFrame(data, 
                           columns=["Unit", "Min value", "Max value", "Description"],
                           index = ["c_max" , "beta" , "alpha", "K_s", "K_q"])
model_param

,Unit,Min value,Max value,Description
c_max,mm,1.00,100.00,Maximum storage capacity
beta,-,0.00,2.00,Degree of spatial variability of the c_max
alpha,-,0.01,0.99,Factor distributing slow and quick flows
K_s,-,0.01,0.10,Fractional discharge of the slow release reser...
K_q,-,0.10,0.99,Fractional discharge of the quick release rese...


The basin is composed by a group of storages that follow a cumulative distribution F(C) where C is the value of the storage which can vary from 0 to Cmax.

<left><img src="util/HyMOD_diagram.png" width="700px">
    
## Interactive manual calibration
### Simulation time range

In [3]:
T = 120 # days
dates = pd.date_range(start = '2000-01-01', periods = T)

### Inputs

In [4]:
P = 20 * np.random.random(T)
PET = 5 * np.random.random(T)
init = [20,20,10,10,10]

### Function to update the Hymod simulation when changing the parameters with the sliders

In [5]:
def update_sim(c_max, beta, alpha, K_s, K_q):
    model = hymod(c_max.value, beta.value, alpha.value, K_s.value, K_q.value)
    ER, Q_q, Q_s, Q = model.simulation(P, PET, init)
    ER_q = ER*alpha.value
    ER_s = ER*(1-alpha.value)
    return ER, ER_q, ER_s, Q_q, Q_s, Q

### Function to update the figure when changing the parameters with the sliders

In [6]:
def update_figure(change):
    with fig_hyd.batch_animate(duration=1000):
        fig_hyd.data[1].y = update_sim(c_max, beta, alpha, K_s, K_q)[5]
    with fig_sto.batch_animate(duration=1000):
        fig_sto.data[0].x = 1 - (1 - np.arange(0,c_max.value+1,1)/c_max.value)**beta.value
        fig_sto.data[0].y = np.arange(0,c_max.value+1,1)
        fig_sto.data[1].line.width = update_sim(c_max, beta, alpha, K_s, K_q)[0][0]
        fig_sto.data[2].marker.size = update_sim(c_max, beta, alpha, K_s, K_q)[0][0]*3
    with fig_flo.batch_animate(duration=1000):
        fig_flo.data[0].line.width = update_sim(c_max, beta, alpha, K_s, K_q)[1][0]
        fig_flo.data[1].marker.size = update_sim(c_max, beta, alpha, K_s, K_q)[1][0]*3
        fig_flo.data[2].line.width = update_sim(c_max, beta, alpha, K_s, K_q)[2][0]
        fig_flo.data[3].marker.size = update_sim(c_max, beta, alpha, K_s, K_q)[2][0]*3
        fig_flo.data[6].line.width = np.mean(update_sim(c_max, beta, alpha, K_s, K_q)[3])
        fig_flo.data[7].marker.size = np.mean(update_sim(c_max, beta, alpha, K_s, K_q)[3])*3
        fig_flo.data[8].line.width = np.mean(update_sim(c_max, beta, alpha, K_s, K_q)[4])
        fig_flo.data[9].marker.size = np.mean(update_sim(c_max, beta, alpha, K_s, K_q)[4])*3        

### Definition of the sliders
#### c_max: Maximum soil moisture storage capacity (mm)

In [7]:
c_max = widgets.FloatSlider(min=model_param.loc['c_max','Min value'],
                            max=model_param.loc['c_max','Max value'],
                            value=50, step = 1,
                            description = 'c_max: Maximum soil moisture storage capacity (mm)',
                            continuous_update=False,
                          style = {'description_width': '350px'} ,layout={'width': '700px'})
c_max.observe(update_figure,names = 'value')

#### beta: Degree of spatial variability of the c_max

In [8]:
beta = widgets.FloatSlider(min=model_param.loc['beta','Min value'],
                           max=model_param.loc['beta','Max value'],
                           value=1, step = 0.01,
                           description = 'beta: Degree of spatial variability of the c_max',
                           continuous_update=False,
                          style = {'description_width': '350px'} ,layout={'width': '700px'})
beta.observe(update_figure,names = 'value')

#### alpha: Factor distributing slow and quick flows

In [9]:
alpha = widgets.FloatSlider(min=model_param.loc['alpha','Min value'],
                            max=model_param.loc['alpha','Max value'],
                            value=0.5, step = 0.01, 
                            description = 'alpha: Factor distributing slow (s) and quick (q) flows (q/s)',
                            continuous_update=False,
                          style = {'description_width': '350px'} ,layout={'width': '700px'})
alpha.observe(update_figure,names = 'value')

#### K_s: Fractional discharge of the slow release reservoir

In [10]:
K_s = widgets.FloatSlider(min=model_param.loc['K_s','Min value'],
                          max=model_param.loc['K_s','Max value'],
                          value=0.05, step = 0.01, 
                          description = 'K_s: Fractional discharge of slow release reservoir',
                          continuous_update=False,
                          style = {'description_width': '350px'} ,layout={'width': '700px'})
K_s.observe(update_figure,names = 'value')

#### K_q: Fractional discharge of the quick release reservoir

In [11]:
K_q = widgets.FloatSlider(min=model_param.loc['K_q','Min value'],
                          max=model_param.loc['K_q','Max value'],
                          value=0.55, step = 0.01,
                          description = 'K_q: Fractional discharge of quick release reservoir',
                          continuous_update=False,
                          style = {'description_width': '350px'} ,layout={'width': '700px'})
K_q.observe(update_figure,names = 'value')

### Observed hydrograph

In [12]:
c_max_obs = np.random.uniform(model_param.loc['c_max','Min value'], model_param.loc['c_max','Max value'])
beta_obs  = np.random.uniform(model_param.loc['beta','Min value'],  model_param.loc['beta','Max value'])
alpha_obs = np.random.uniform(model_param.loc['alpha','Min value'], model_param.loc['alpha','Max value'])
K_s_obs   = np.random.uniform(model_param.loc['K_s','Min value'],   model_param.loc['K_s','Max value'])
K_q_obs   = np.random.uniform(model_param.loc['K_q','Min value'],   model_param.loc['K_q','Max value'])

In [13]:
model_obs = hymod(c_max_obs, beta_obs, alpha_obs, K_s_obs, K_q_obs)
ER_obs, Q_q_obs,Q_s_obs,Q_obs = model_obs.simulation(P, PET, init)

### Plot the interactive figure
#### Initial simulation

In [14]:
model = hymod(c_max.value, beta.value, alpha.value, K_s.value, K_q.value)
ER_sim, Q_q_sim,Q_s_sim, Q_sim = model.simulation(P, PET, init)
ER_q_sim = ER_sim * alpha.value
ER_s_sim = ER_sim * (1-alpha.value)
c = np.arange(0,c_max.value,1)
F_c = 1 - (1 - c/c_max.value)**beta.value

In [15]:
np.mean(Q_q_sim)
np.mean(Q_s_sim)

3.3914438913229623

#### Figure: hydrographs (with two traces: simulated and observed hydrogrphs)

In [16]:
sim_hyd = go.Scatter(x=dates, y=Q_sim, name='sim hyd')
obs_hyd = go.Scatter(x=dates, y=Q_obs, name='obs hyd')
fig_hyd = go.FigureWidget(data   = [obs_hyd,sim_hyd],
                          layout = go.Layout(xaxis = dict(title = 'date'),
                                             yaxis = dict(range = [0,20],title = 'Q')))

#### Figure: storage capacity

In [17]:
sto_trace = go.Scatter(x=F_c, y=c, name=None, fill="tozeroy", fillcolor = 'sandybrown',mode='none')
ER_line = go.Scatter(x=[0.01,0.4], y=[50,50], mode = 'lines', line = dict(color = 'blue', width = ER_sim[0]),opacity = 0.9)
ER_head = go.Scatter(x=[-0.01], y=[50], mode = 'markers', opacity = 0.9, marker = dict(symbol = 'triangle-right', size = ER_sim[0]*3, color = 'blue'))
sto_layout = go.Layout(xaxis = dict(title = 'F(c)',autorange='reversed',range = [1,0], showgrid = False),
                       yaxis = dict(range = [0,model_param.loc['c_max','Max value']],
                                    title = 'mm', showgrid = False),
                       width=350, height=350, plot_bgcolor=None, showlegend=False,
                       annotations = [dict(x = 0.55, y = 5, text = 'Soil storage capacity',showarrow=False),
                                      dict(x = 0.3, y = 60, text = 'Effective rain',showarrow=False)])
fig_sto = go.FigureWidget(data   = [sto_trace,ER_line,ER_head],
                          layout = sto_layout)

#### Figure: quick and slow flow

In [18]:
ER_q_line = go.Scatter(x=[0,5], y=[8,8], mode = 'lines', line = dict(color = 'blue', width = ER_q_sim[0]),opacity = 0.7)
ER_q_head = go.Scatter(x=[5],   y=[8], mode = 'markers', opacity = 0.7,marker = dict(symbol = 'triangle-right', size = ER_q_sim[0]*3, color = 'blue'))
ER_s_line = go.Scatter(x=[0,5], y=[7.5,3], mode = 'lines', line = dict(color = 'blue', width = ER_s_sim[0]),opacity = 0.4)
ER_s_head = go.Scatter(x=[5],   y=[3], mode = 'markers', opacity = 0.4,marker = dict(symbol = 'triangle-se', size = ER_s_sim[0]*3, color = 'blue'))

res_q = go.Scatter(x=[6,14], y=[8,8], mode = 'lines', line = dict(color = 'blue', width = 80),opacity = 0.7)
res_s = go.Scatter(x=[6,14], y=[3,3], mode = 'lines', line = dict(color = 'blue', width = 80),opacity = 0.4)

Q_q_line = go.Scatter(x=[15,20], y=[8,8], mode = 'lines', line = dict(color = 'blue', width = np.mean(Q_q_sim)),opacity = 0.7)
Q_q_head = go.Scatter(x=[20],   y=[8], mode = 'markers', opacity = 0.7,marker = dict(symbol = 'triangle-right', size = np.mean(Q_q_sim)*3, color = 'blue'))
Q_s_line = go.Scatter(x=[15,20], y=[3,7.5], mode = 'lines', line = dict(color = 'blue', width = np.mean(Q_s_sim)),opacity = 0.4)
Q_s_head = go.Scatter(x=[20],   y=[7.5], mode = 'markers', opacity = 0.4,marker = dict(symbol = 'triangle-ne', size = np.mean(Q_s_sim)*3, color = 'blue'))

flo_layout = go.Layout(width=650, height=450,plot_bgcolor='white',showlegend=False,
                       xaxis = dict(range = [0,22],showticklabels=False),
                       yaxis = dict(range = [0,10],showticklabels=False),
                       annotations = [dict(x = 2, y = 9, text = 'quick EC',showarrow=False),
                                      dict(x = 10, y = 9, text = 'quick reservoir',showarrow=False),
                                      dict(x = 18, y = 9, text = 'quick flow',showarrow=False),
                                      dict(x = 2, y = 4, text = 'slow EC',showarrow=False),
                                      dict(x = 10, y = 4, text = 'slow reservoir',showarrow=False),
                                      dict(x = 18, y = 4, text = 'slow flow',showarrow=False)])

fig_flo = go.FigureWidget(data   = [ER_q_line,ER_q_head,ER_s_line,ER_s_head,res_q,res_s,
                                    Q_q_line,Q_q_head,Q_s_line,Q_s_head],
                          layout = flo_layout)

#### Plot

In [19]:
widgets.VBox([widgets.VBox([c_max,beta,alpha,K_s, K_q]),
              widgets.HBox([fig_sto,fig_flo]),fig_hyd])